In [4]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()


llm = ChatGroq(
     model="llama-3.1-8b-instant",
     temperature=0
)

response = llm.invoke("What is the capital of sri lanka?")
print(response.content)



The capital of Sri Lanka is Colombo. However, the administrative capital is Sri Jayawardenepura Kotte.


In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are an intelligent exam marks evaluator who has a speciality in mathematics.

You will receive:
1. OCR text extracted from a student's answer sheet.
2. The original correct answer for reference.
3. The maximum marks possible.

Your task:
- Carefully evaluate the student's OCR answer.
- Do NOT assume missing words or unclear OCR content.
- Do NOT infer anything that is not clearly written.
- Do NOT do mistake when totaling the awarded marks
- Decide step-wise marks allocation on your own.
- Award marks strictly based on visible valid mathematical steps.

Output Format (STRICT JSON):
{{
  "evaluation": [
    {{
      "step": "Step description",
      "correct": true/false,
      "marks_awarded": number
    }}
  ],
  "total_obtained_marks": number,
  "maximum_marks": number
}}
"""
        ),
        (
            "human",
            "Here is the OCR text: {OCR_text}\n\n"
            "Here is the original answer for reference: {original_answer}\n\n"
            "Maximum marks possible: {max_marks}"
        ),
    ]
)


In [25]:
ocr_text = r"""
  $1 \sin (45-30)$
        $$
        \begin{aligned}
        & =\sin 45 \cdot \cos 30-\cos 45 \sin 30 \\
        & =\frac{1}{\sqrt{2}} \times \frac{\sqrt{3}}{2}-\frac{1}{\sqrt{2}} \times \frac{1}{2} \\
        & =\frac{\sqrt{6}-\sqrt{2}}{4}
        \end{aligned}
        $$
      
"""

original_answer = r"""
(1) sin(45° − 30°)

Using identity:
sin(A − B) = sinA cosB − cosA sinB

= sin45° cos30° − cos45° sin30°
= (1/√2)(√3/2) − (1/√2)(1/2)
= √3/(2√2) − 1/(2√2)
= (√6 − √2)/4

"""

max_marks = "5"

In [26]:
#   (2) $3^{x}=12$
#         $$
#         \begin{aligned}
#         & \Rightarrow \log 3^{x}=\log 12 \\
#         & \Rightarrow x \log 3=\log 12 \\
#         & \Rightarrow x=\log 12 / \log 3 \\
#         & \Rightarrow x=2.26
#         \end{aligned}
#         $$

In [27]:
chain = prompt | llm
response = chain.invoke(
    {
        "OCR_text": ocr_text,
        "original_answer": original_answer,
        "max_marks": max_marks
    }
)

print(response.content)


{
  "evaluation": [
    {
      "step": "Apply the sin(A - B) identity",
      "correct": true,
      "marks_awarded": 1
    },
    {
      "step": "Correctly expand the identity",
      "correct": true,
      "marks_awarded": 1
    },
    {
      "step": "Correctly calculate the first term",
      "correct": true,
      "marks_awarded": 1
    },
    {
      "step": "Correctly calculate the second term",
      "correct": false,
      "marks_awarded": 0
    },
    {
      "step": "Correctly simplify the expression",
      "correct": true,
      "marks_awarded": 1
    }
  ],
  "total_obtained_marks": 4,
  "maximum_marks": 5
}


In [28]:
ocr_text = r"""
(2) 3^x = 12

Taking logarithm on both sides:

log(3^x) = log(12)
x log 3 = log 12
x log 3 = log 12
x log 3 = log 12
x log 3 = log 12

x ≈ 2.26
"""

original_answer = r"""
(2) 3^x = 12

Taking logarithm on both sides:

log(3^x) = log(12)
x log 3 = log 12
x = log 12 / log 3
Using common logarithm values:
log 12 ≈ 1.07918
log 3 ≈ 0.47712
x ≈ 1.07918 / 0.47712
x ≈ 2.26
"""

max_marks = 10  # integer, not string



# --- Step 4: Create chain and evaluate ---
chain = prompt | llm

response = chain.invoke(
    {
        "OCR_text": ocr_text,
        "original_answer": original_answer,
        "max_marks": max_marks
    }
)

# --- Step 5: Print output ---
print(response.content)

{
  "evaluation": [
    {
      "step": "Taking logarithm on both sides of the equation 3^x = 12",
      "correct": true,
      "marks_awarded": 2
    },
    {
      "step": "Applying the logarithm property to simplify log(3^x) = log(12) to x log 3 = log 12",
      "correct": true,
      "marks_awarded": 2
    },
    {
      "step": "Correctly expressing x in terms of logarithms as x = log 12 / log 3",
      "correct": false,
      "marks_awarded": 0
    },
    {
      "step": "Using common logarithm values to calculate log 12 and log 3",
      "correct": true,
      "marks_awarded": 2
    },
    {
      "step": "Correctly calculating x ≈ 1.07918 / 0.47712",
      "correct": true,
      "marks_awarded": 2
    },
    {
      "step": "Correctly approximating x ≈ 2.26",
      "correct": true,
      "marks_awarded": 2
    }
  ],
  "total_obtained_marks": 8,
  "maximum_marks": 10
}
